# Part 1 - Exploring Datasets

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
df = pd.read_csv('HW1_data.csv')

## Question 1
**How many entries does the dataset contain?**

**Answer:**

In [2]:
df.shape

(684, 11)

The dataset contains 684 different data entry points.

## Question 2
**Which fields in the dataset are 'numerical' and which are 'categorical'?**

**Answer**:


First, let us view what type of columns the dataframe generated from the dataset contains.

In [15]:
df.head(0)

,league,year,position,team,matches,wins,draws,loses,scored,conceded,pts


The **numerical fields** in the data set are: matchs, wins, draws, loses , scored , conceded ,pts (points).

while the **categorical fields** in the given data set are: league, year , position , team.

Although position and year are numerical values. It is evident they can be used to categorize our data as they give
us clear cutoffs 'who was first place?' 

'in the year x which team played' as opposed to the numerical fields which
aren't continious unless we arbitrerly define certain conditions which aren't defined naturally.

## Question 3
**How many unique values does each one of the categorical contain?**

**Answer:**

In [4]:
categorical = ["league", "year" , "position" , "team"]
for category in categorical:
    
    #type(field)
    print("Field","'"+category+"'" , "has", len(df[category].unique()), "unique values.")


Field 'league' has 6 unique values.
Field 'year' has 6 unique values.
Field 'position' has 20 unique values.
Field 'team' has 168 unique values.


## Question 4
**Which fields in the dataset contain missing values (such as NULL) and how many are there per field?**

Answer:

In [5]:
print(df.isna().sum())

league      0
year        0
position    0
team        0
matches     0
wins        0
draws       0
loses       0
scored      0
conceded    3
pts         0
dtype: int64


In the current data set the only field with missing values is the field 'conceded' and it has 3 missing values

## Question 5
**How many different teams played for each one of the league over the years in the dataset? Which league included the least amount of unique teams?**

**Answer:**

In [6]:
gb_league = df.groupby('league')

gb_league['team'].unique().apply(len)

league
Bundesliga    24
EPL           30
La_liga       30
Ligue_1       29
RFPL          25
Serie_A       30
Name: team, dtype: int64

The Bundesliga league had the least amount of teams with only 24 teams

## Question 6

**Present the average goal count each time had for each year. In addition, show the linear correlation between the average goal count for each team to their respective placement in the league. Explain the significance of your findings.**

**Answer**

In [7]:
meanScore_df =  {                  #I would rather not submit this incredibly jank but it does work regardless of the data
    'year' : [2014],
    'position': [1],
    'mean_goal_per_match' : [3.2],            #created a dictionary to house the final data 
                                #note : if a team played more than once a year we would not be able to just splice the 
}                               #original data frame as we would have to make sure to avg out ALL the games the team played
                                #and house them only once in the dataframe used for the correlation
meanScore_df = pd.DataFrame(meanScore_df)
meanScore_df = meanScore_df.drop(index=0) 
team_list = df['team'].unique()
gb_team = df.groupby('team')

for team in team_list:                          #going over each team
    certain_team = gb_team.get_group(team)
    unique_years = (certain_team['year'].unique())
    certain_year = certain_team.groupby('year')
    
    for years in unique_years:                 #going over each year a certain team had played
        group = certain_year.get_group(years)
        #if group.shape[0] > 1:                           #these are the lines that were used to check if any team played
        #print (group.shape[0])                           #more than once a year
        means = group['scored'].sum() / group['matches'].sum()   #counting the mean of ALL the games of a certain year
        tmp =  {                                                #placing said data in a dictionary
        'year' : [years],
        'position': [group.iloc[0,2]],
        'mean_goal_per_match' : [means],
        }
        tmp = pd.DataFrame(tmp)                                 #converting and concating each dictionary 
        meanScore_df = pd.concat([meanScore_df, tmp],ignore_index=True)
        
        
meanScore_df

,year,position,mean_goal_per_match
0,2014,1,2.894737
1,2015,1,2.947368
2,2016,2,3.052632
3,2017,1,2.605263
4,2018,1,2.368421
...,...,...,...
679,2017,15,0.766667
680,2017,16,0.533333
681,2018,16,0.800000
682,2019,12,1.333333


In [8]:
print("the correlation is" ,meanScore_df['position'].corr(meanScore_df['mean_goal_per_match']))

the correlation is -0.7587818572237598


As expected we have found that there is a strong correlation between the mean goals to the placement of a team for a league. Such a correlation suggests that the more a team scores the more likely it is for that them to also be placed high. Such results are not suprising as the score that the team

## Question 7
**Present for each year and each league seperately the mean goal count of each team.Based on the output you produced which league had the biggest discrepency between the biggest mean in goal count of the years to the lowest mean of the years.**
**Which year had the lowest mean goal count and which year had the highest goal count**

missing

## Question 8
**The points each team recieved is calculated in the following manner:**

**Victory in a game is worth 3 points.**

**Tie in a game is worth 1 point.**

**Defeat in a game is worth 0 points.**

**The column 'pts' is supposed to represent such calculation, does it appear there's an error? If so for which league, which league and which team?
In your opinion does the fault originate from the 'pts' column? Or one of the other columns used to calculate the points?**

**Answer**

In [9]:
index = df.index # getting index of the dataframe
missmatch_array = [] #containing 
for i in index:
    row = df.iloc[i]
    calculation = 3*row.wins + row.draws
    if calculation != row.pts:
        tmp = [i]
        missmatch_array.extend(tmp)
missmatch_array

[425]

We have found only a single row that has a mismatch between the expected result from the calculation to the 'pts' column and that is:

In [10]:
df.loc[425]

league      Serie_A
year           2017
position         18
team        Crotone
matches          38
wins              9
draws             8
loses            21
scored           40
conceded       66.0
pts             180
Name: 425, dtype: object

As every other entry we checked for hasn't had any discrepency between the supposed calculation and the 'pts' value it should have. It would be fair to conclude the difference resulted due to faulty data. In other words the person and or machine that had organized the csv file made an error while inputting the values.

## Question 9

**For this question we will define "Failure" as a team that conceded more goals in a certain year than the goals it scored
assuming teams that had a missing value (NULL) for their conceded goals for a  certain year which league had the biggest amount of "Failure" teams over the years?**

**Answer**

In [11]:
filtered_df = df[df.conceded.isna() == False]
filtered_df.reset_index(inplace = True) #filter out all the teams with null conceded 

leagues = filtered_df['league'].unique()
league_counterdf = { league: 0 for league in leagues}#creating a dictionary to house the amount of "loser" teams per league


years = filtered_df['year'].unique() #list of unique years
years_split = filtered_df.groupby('year') #splitting based on the years

for year in years:  #for every year we are going to split again per team to find out if that team is a loser team
    certain_year = years_split.get_group(year)
    teams = certain_year['team'].unique()
    teams_split = certain_year.groupby('team')
    for team in teams: 
        certain_team = teams_split.get_group(team)
            
        if certain_team['scored'].sum() < certain_team['conceded'].sum(): #checking based on a certain year and team
                                                                          #if they are a "Loser" team
            certain_team.reset_index(inplace=True)
            for i in range(certain_team.shape[0]):
                
                loser_team_league = certain_team.loc[i,'league']          #adding a count to the dictionary containing
                league_counterdf[loser_team_league] +=1                   #the amount of losers it had in total
                
league_counterdf = pd.DataFrame([league_counterdf])
league_counterdf.apply(sum,axis='rows')              #using sum doesn't change the data just used to make the presentation 
                                                     #cleaner

La_liga       75
EPL           71
Bundesliga    63
Serie_A       63
Ligue_1       69
RFPL          52
dtype: int64

# Part 2 - Data Representation

## Question 1 - Top Six Countries by Military Expenditures

### Sub-Question A
**What could you infer about the military expenditure of the different countries in the world over the years?**

**Answer:**

Given the graph shows the amount of US dollars spent on the military budget over the years and we can clearly see most countries seem to be trending in increasing their military budget especially east/middle eastern countries

### Sub-Question B
**Point to an issue with the graph that might make someone reach the wrong conclusions**

**Answer**:

The author attempted to represnt multiple countries in the same graph. However, based on the axes we can see that the US  infact uses a different scaling for the points in its graph. At first glance, it might look as if China has increased its budget past the US even though in reality its budget is nowhere near the US enormous budget.

### Sub-Question C -Image MISSING
**Suggest an alternative visualization that will represent the data in a more effective manner**

**Answer**:

In [12]:
#Question C
print("Part B-1 Question C")
#Include image

Part B-1 Question C


## Question 2 - Glyphosate and GE to Death Rate

### Sub-Question A

**What could you infer on the relationship between 'deaths from obesity' and the usage of Glyphosate and GE in the corn and soy industry?**

**Answer**:

The graph depicits the death rate before and after the usage of certain chemicals on foods. We can observe that there had been a dramatic increase on the count of deaths after the introduction of said chemicals into the food industry. Such trend suggests there's a correlation between the usage of the chemicals and the deaths that occured over the years

### Sub-Question B
**While designing the graph some unconventional measures were taken in order to reperesent a certain message. In your opinion, what is the message the the graph's creator attempted to convey? Do you agree with said message?**

**Answer:**

It is clear the author attempted to claim there is a correlational/ causational relationship between the death rate and the addition of the chemicals. A correlational relationship does seem likely as just as GE and glyphosate had been added there had been a large spike in the death toll. However, this data alone does not firmly conclude there is a causational relationship as there might be an alternative explanation as to why there had been an increase of deaths.

### Sub_Question C -- INCOMPLETE
**Find in the visualization two issues which might detract from its effectivness. Explain said issues**

**Answer**:

There are two glaring issues with the the graph:

**Axes**: the graphs axes aren't reset to start at 0 rather they start a different point making the graph 'zoomed in'. Such alternation gives the appereance that the changes are more dramatic than they are in reality. In particulary the  'Deaths per 100,000' axe 

""

## Question 3 - Global Vegan Cheese Market Graph

### Sub-Question A
**What is the information that is expected to be conveyed using this infograph?**


**Answer**:

The graph attempts to show the growth of the 'Global Cheese Vegan Market' by showing a pie chart depicting the share each year contributed to the total netshare of the cheese market and it mentions the average growth that occured over the years

### Sub-Question B
**Find in the visualization at least 3 issues that might make it less effective. Explain said issues**

**Answer**:

There are three main glaring issues with the infograph:

(1) **Pie chart**: the usage of pie chart makes discerning the difference in the market share per year harder as we are unable to tell the exact difference of values between the years

(2) **Labels**: The given chart doesn't detail which slice belongs to which year.Moreover, the CAGR shows the average growth from 2019 to 2026. However, one of the slices is labeled as 2018, this raises the question whether or not the other slices depict the market size for other years within said range.

(3) **Background**: the background given in this infograph doesn't contribute any information as we can already see based on the title we are looking at the market share of vegan cheese. The addition of the image creates unnecessary clutter and should be avoided whenever possible.


### Sub-Question C -- MISSING
**Suggest an alternative visualzation that would represent the data in a clear and more effective manner**

## Question 4 -- MISSING
What is the issue with the tittle "Average North Carolina teacher pay is nearly $ 58,000, state says. But educators argue many earn lesss."
Answer:

As we discussed in the lectures, the average is sensitive to extreme values. The title suggest that it is a contradiction beteween the actual (undisclosed in the title) salary of many teachers and the average represented. However, we would and should expect "many" (which by itself is ambigious) to earn less than the average and "many" to earn more 